In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok='\033[92m'
    fail='\033[91m'
    close='\033[0m'

In [3]:
TRAINING_SIZE=80000
DIGITS=3
REVERSE=False
MAXLEN=DIGITS+1+DIGITS
chars='0123456789+ '
RNN=layers.LSTM
HIDDEN_SIZE=128
BATCH_SIZE=128
LAYERS=1

In [4]:
class CharacterTable(object):
    def __init__(self,chars):
        self.chars=sorted(set(chars))
        self.char_indices=dict((c,i) for i,c in enumerate(self.chars))
        self.indices_char=dict((i,c) for i,c in enumerate(self.chars))
    
    def encode(self,C,num_rows): #input C is string
        x=np.zeros((num_rows, len(self.chars)))
        for i,c in enumerate(C):
            x[i,self.char_indices[c]]=1
        return x
    
    def decode(self,x,calc_argmax=True): #if input x is 1-D array then culc_argmax=True； otherwise, if x is string or int then False
        if calc_argmax:
            x=x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)
    

In [5]:
ctable=CharacterTable(chars)

In [6]:
print(ctable.indices_char)
print(ctable.char_indices)

{0: ' ', 1: '+', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9'}
{' ': 0, '+': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11}


# Data Generation

In [7]:
questions=[]
expected=[]
seen=set()
print('Generating data....')
while len(questions)<TRAINING_SIZE:
    f=lambda:int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1,DIGITS+1))))
    a,b=f(),f()
    key=tuple(sorted((a,b)))
    if key in seen:
        continue
    seen.add(key)
    q='{}+{}'.format(a,b)
    query=q+' '*(MAXLEN-len(q))
    ans=str(a+b)
    ans+=' '*(DIGITS+1-len(ans))
    if REVERSE:
        query=query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:',len(questions))

Generating data....
Total addition questions: 80000


In [8]:
print(questions[:5],expected[:5])

['5+68   ', '77+505 ', '87+702 ', '1+912  ', '9+52   '] ['73  ', '582 ', '789 ', '913 ', '61  ']


# Processing

In [9]:
print('Vectorization....')
x=np.zeros((len(questions),MAXLEN,len(chars)),dtype=np.bool)
y=np.zeros((len(expected),DIGITS+1,len(chars)),dtype=np.bool)
for i,sentence in enumerate(questions):
    x[i]=ctable.encode(sentence,MAXLEN)
for i,sentence in enumerate(expected):
    y[i]=ctable.encode(sentence,DIGITS+1)

Vectorization....


In [10]:
indices=np.arange(len(y))
np.random.shuffle(indices)
x=x[indices]
y=y[indices]

#train_test_split
train_x=x[:20000]
train_y=y[:20000]
test_x=x[20000:]
test_y=y[20000:]

split_at=len(train_x)-len(train_x)//10
(x_train,x_val)=train_x[:split_at],train_x[split_at:]
(y_train,y_val)=train_y[:split_at],train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Date:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Date:
(60000, 7, 12)
(60000, 4, 12)


In [11]:
print('input: ',x_train[:3],'\n\n','label: ',y_train[:3])

input:  [[[False False False False False False False False False False  True
   False]
  [False False False False False False  True False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False False False
    True]
  [False False False False False False  True False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False  True False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [False False False False False False Fal

# Build Model

In [12]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [13]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 278us/step - loss: 2.0119 - acc: 0.2930 - val_loss: 1.8637 - val_acc: 0.3313
Q 1+880   T 881  ☒ 100 
Q 207+354 T 561  ☒ 110 
Q 307+74  T 381  ☒ 100 
Q 8+875   T 883  ☒ 100 
Q 90+814  T 904  ☒ 100 
Q 815+721 T 1536 ☒ 110 
Q 268+895 T 1163 ☒ 1101
Q 220+71  T 291  ☒ 100 
Q 365+975 T 1340 ☒ 110 
Q 45+231  T 276  ☒ 100 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 147us/step - loss: 1.8537 - acc: 0.3294 - val_loss: 1.8273 - val_acc: 0.3332
Q 981+57  T 1038 ☒ 108 
Q 630+128 T 758  ☒ 108 
Q 203+18  T 221  ☒ 128 
Q 482+59  T 541  ☒ 108 
Q 176+16  T 192  ☒ 108 
Q 933+99  T 1032 ☒ 108 
Q 62+507  T 569  ☒ 108 
Q 539+16  T 555  ☒ 108 
Q 588+898 T 1486 ☒ 1

18000/18000 [==============================] - 3s 147us/step - loss: 1.2280 - acc: 0.5429 - val_loss: 1.2191 - val_acc: 0.5426
Q 12+55   T 67   ☑ 67  
Q 608+629 T 1237 ☒ 1215
Q 823+74  T 897  ☒ 810 
Q 49+272  T 321  ☒ 315 
Q 701+115 T 816  ☒ 821 
Q 9+46    T 55   ☒ 58  
Q 521+396 T 917  ☒ 902 
Q 575+3   T 578  ☒ 500 
Q 50+958  T 1008 ☒ 101 
Q 381+359 T 740  ☒ 722 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 146us/step - loss: 1.1919 - acc: 0.5560 - val_loss: 1.1860 - val_acc: 0.5575
Q 436+899 T 1335 ☑ 1335
Q 693+65  T 758  ☒ 730 
Q 430+544 T 974  ☒ 906 
Q 41+254  T 295  ☒ 390 
Q 265+165 T 430  ☒ 496 
Q 69+451  T 520  ☒ 501 
Q 726+881 T 1607 ☒ 1610
Q 912+214 T 1126 ☒ 1103
Q 79+448  T 527  ☒ 542 
Q 16+0    T 16   ☒ 28  

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 147us/step - loss: 0.7462 - acc: 0.7199 - val_loss: 0.7599 - val_acc: 0.7029
Q 827+841 T 1668 ☒ 1660
Q 222+288 T 510  ☒ 500 
Q 6+448   T 454  ☑ 454 
Q 470+24  T 494  ☒ 498 
Q 88+234  T 322  ☑ 322 
Q 493+75  T 568  ☑ 568 
Q 447+47  T 494  ☑ 494 
Q 300+73  T 373  ☒ 363 
Q 509+59  T 568  ☒ 567 
Q 0+197   T 197  ☒ 198 

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 146us/step - loss: 0.6731 - acc: 0.7503 - val_loss: 0.6864 - val_acc: 0.7375
Q 53+10   T 63   ☒ 53  
Q 96+4    T 100  ☑ 100 
Q 95+805  T 900  ☒ 899 
Q 262+57  T 319  ☒ 329 
Q 710+98  T 808  ☒ 899 
Q 404+978 T 1382 ☒ 1390
Q 308+8   T 316  ☑ 316 
Q 62+191  T 253  ☑ 253 
Q 62+967  T 1029 ☒ 1020
Q 989+41  T 1030 ☑ 1030

--------------------------------------------------
Iteration 30
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 146us/step - loss: 0.1856 - acc: 0.9630 - val_loss: 0.2274 - val_acc: 0.9333
Q 119+1   T 120  ☒ 110 
Q 73+343  T 416  ☑ 416 
Q 186+677 T 863  ☑ 863 
Q 277+41  T 318  ☑ 318 
Q 249+13  T 262  ☑ 262 
Q 575+3   T 578  ☑ 578 
Q 0+777   T 777  ☑ 777 
Q 385+43  T 428  ☑ 428 
Q 1+946   T 947  ☑ 947 
Q 953+64  T 1017 ☑ 1017

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 149us/step - loss: 0.1674 - acc: 0.9690 - val_loss: 0.1974 - val_acc: 0.9469
Q 320+338 T 658  ☑ 658 
Q 13+236  T 249  ☑ 249 
Q 330+383 T 713  ☑ 713 
Q 28+501  T 529  ☑ 529 
Q 779+7   T 786  ☑ 786 
Q 35+908  T 943  ☑ 943 
Q 200+629 T 829  ☒ 839 
Q 514+579 T 1093 ☑ 1093
Q 336+9   T 345  ☒ 355 
Q 14+98   T 112  ☑ 112 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 158us/step - loss: 0.0549 - acc: 0.9937 - val_loss: 0.0922 - val_acc: 0.9736
Q 879+820 T 1699 ☑ 1699
Q 16+850  T 866  ☑ 866 
Q 419+249 T 668  ☑ 668 
Q 27+704  T 731  ☑ 731 
Q 32+41   T 73   ☑ 73  
Q 971+97  T 1068 ☑ 1068
Q 6+710   T 716  ☑ 716 
Q 51+48   T 99   ☑ 99  
Q 53+123  T 176  ☑ 176 
Q 539+336 T 875  ☑ 875 

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 146us/step - loss: 0.0533 - acc: 0.9931 - val_loss: 0.1036 - val_acc: 0.9683
Q 442+20  T 462  ☑ 462 
Q 6+225   T 231  ☑ 231 
Q 71+996  T 1067 ☑ 1067
Q 923+867 T 1790 ☒ 1780
Q 42+406  T 448  ☑ 448 
Q 864+57  T 921  ☑ 921 
Q 186+24  T 210  ☑ 210 
Q 23+731  T 754  ☑ 754 
Q 66+836  T 902  ☑ 902 
Q 98+760  T 858  ☑ 858 

--------------------------------------------------
Iteration 58
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 151us/step - loss: 0.0797 - acc: 0.9766 - val_loss: 0.1216 - val_acc: 0.9647
Q 941+79  T 1020 ☒ 1010
Q 392+944 T 1336 ☑ 1336
Q 950+869 T 1819 ☒ 1829
Q 233+66  T 299  ☑ 299 
Q 32+41   T 73   ☑ 73  
Q 22+515  T 537  ☑ 537 
Q 92+293  T 385  ☑ 385 
Q 693+65  T 758  ☒ 768 
Q 742+390 T 1132 ☑ 1132
Q 448+588 T 1036 ☑ 1036

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 154us/step - loss: 0.0749 - acc: 0.9786 - val_loss: 0.0642 - val_acc: 0.9816
Q 320+56  T 376  ☑ 376 
Q 294+60  T 354  ☑ 354 
Q 617+695 T 1312 ☑ 1312
Q 623+17  T 640  ☑ 640 
Q 970+35  T 1005 ☑ 1005
Q 470+220 T 690  ☑ 690 
Q 243+892 T 1135 ☑ 1135
Q 18+283  T 301  ☑ 301 
Q 647+933 T 1580 ☑ 1580
Q 411+381 T 792  ☒ 892 

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 144us/step - loss: 0.0103 - acc: 0.9994 - val_loss: 0.0372 - val_acc: 0.9900
Q 32+129  T 161  ☑ 161 
Q 1+489   T 490  ☑ 490 
Q 70+319  T 389  ☑ 389 
Q 612+890 T 1502 ☑ 1502
Q 750+7   T 757  ☑ 757 
Q 970+35  T 1005 ☑ 1005
Q 856+916 T 1772 ☑ 1772
Q 85+239  T 324  ☑ 324 
Q 183+463 T 646  ☑ 646 
Q 649+232 T 881  ☑ 881 

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 145us/step - loss: 0.0095 - acc: 0.9995 - val_loss: 0.0369 - val_acc: 0.9890
Q 946+206 T 1152 ☑ 1152
Q 8+197   T 205  ☑ 205 
Q 80+710  T 790  ☑ 790 
Q 57+125  T 182  ☑ 182 
Q 31+585  T 616  ☑ 616 
Q 480+755 T 1235 ☑ 1235
Q 5+291   T 296  ☑ 296 
Q 605+267 T 872  ☑ 872 
Q 82+360  T 442  ☑ 442 
Q 96+763  T 859  ☑ 859 

--------------------------------------------------
Iteration 86
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 151us/step - loss: 0.0056 - acc: 0.9998 - val_loss: 0.0333 - val_acc: 0.9891
Q 129+884 T 1013 ☑ 1013
Q 50+442  T 492  ☑ 492 
Q 58+547  T 605  ☑ 605 
Q 344+183 T 527  ☑ 527 
Q 71+756  T 827  ☑ 827 
Q 281+48  T 329  ☑ 329 
Q 81+812  T 893  ☑ 893 
Q 328+690 T 1018 ☑ 1018
Q 48+905  T 953  ☑ 953 
Q 25+123  T 148  ☑ 148 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 156us/step - loss: 0.0073 - acc: 0.9992 - val_loss: 0.0403 - val_acc: 0.9859
Q 456+958 T 1414 ☑ 1414
Q 30+422  T 452  ☑ 452 
Q 526+13  T 539  ☑ 539 
Q 887+74  T 961  ☑ 961 
Q 565+532 T 1097 ☑ 1097
Q 648+58  T 706  ☒ 606 
Q 639+19  T 658  ☑ 658 
Q 209+834 T 1043 ☒ 1033
Q 31+628  T 659  ☑ 659 
Q 861+652 T 1513 ☑ 1513


# Testing

In [14]:
print("MSG : Prediction")

pred_y=model.predict_classes(test_x,verbose=0)
right=0
for i in range(len(test_x)):
    q=ctable.decode(test_x[i])
    correct=ctable.decode(test_y[i])
    pred=ctable.decode(pred_y[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct==pred:
        right+=1
        print(colors.ok+'☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(pred)
print('accuracy=',(right/len(test_x))*100,'%')

MSG : Prediction
Q 1+693   T 694  ☑ 694 
Q 911+33  T 944  ☑ 944 
Q 69+152  T 221  ☑ 221 
Q 88+106  T 194  ☑ 194 
Q 69+126  T 195  ☑ 195 
Q 85+163  T 248  ☑ 248 
Q 559+65  T 624  ☑ 624 
Q 965+152 T 1117 ☑ 1117
Q 555+879 T 1434 ☑ 1434
Q 927+4   T 931  ☑ 931 
Q 78+868  T 946  ☑ 946 
Q 291+734 T 1025 ☑ 1025
Q 641+8   T 649  ☑ 649 
Q 358+157 T 515  ☑ 515 
Q 122+792 T 914  ☑ 914 
Q 26+931  T 957  ☑ 957 
Q 393+639 T 1032 ☑ 1032
Q 6+403   T 409  ☑ 409 
Q 606+663 T 1269 ☑ 1269
Q 894+160 T 1054 ☑ 1054
Q 5+351   T 356  ☑ 356 
Q 515+58  T 573  ☑ 573 
Q 415+518 T 933  ☑ 933 
Q 3+287   T 290  ☑ 290 
Q 98+974  T 1072 ☑ 1072
Q 141+40  T 181  ☑ 181 
Q 376+300 T 676  ☑ 676 
Q 33+659  T 692  ☑ 692 
Q 954+254 T 1208 ☑ 1208
Q 43+751  T 794  ☑ 794 
Q 357+54  T 411  ☑ 411 
Q 18+774  T 792  ☑ 792 
Q 84+131  T 215  ☑ 215 
Q 95+799  T 894  ☑ 894 
Q 9+109   T 118  ☑ 118 
Q 990+412 T 1402 ☒ 1302
Q 68+972  T 1040 ☑ 1040
Q 151+10  T 161  ☑ 161 
Q 421+322 T 743  ☑ 743 
Q 80+74   T 154  ☑ 154 
Q 28+733  T 761  ☑ 761 

Q 22+106  T 128  ☑ 128 
Q 411+99  T 510  ☑ 510 
Q 62+291  T 353  ☑ 353 
Q 742+385 T 1127 ☑ 1127
Q 906+95  T 1001 ☑ 1001
Q 11+46   T 57   ☑ 57  
Q 260+178 T 438  ☑ 438 
Q 227+37  T 264  ☑ 264 
Q 30+852  T 882  ☑ 882 
Q 63+177  T 240  ☑ 240 
Q 915+74  T 989  ☑ 989 
Q 5+579   T 584  ☑ 584 
Q 792+84  T 876  ☑ 876 
Q 1+630   T 631  ☑ 631 
Q 80+17   T 97   ☒ 98  
Q 76+13   T 89   ☑ 89  
Q 440+464 T 904  ☒ 804 
Q 0+804   T 804  ☑ 804 
Q 702+22  T 724  ☑ 724 
Q 60+862  T 922  ☑ 922 
Q 106+96  T 202  ☒ 102 
Q 430+608 T 1038 ☑ 1038
Q 46+550  T 596  ☑ 596 
Q 286+830 T 1116 ☑ 1116
Q 600+3   T 603  ☑ 603 
Q 170+966 T 1136 ☑ 1136
Q 741+13  T 754  ☑ 754 
Q 483+941 T 1424 ☑ 1424
Q 392+74  T 466  ☑ 466 
Q 208+388 T 596  ☑ 596 
Q 76+296  T 372  ☑ 372 
Q 27+618  T 645  ☑ 645 
Q 89+438  T 527  ☑ 527 
Q 4+323   T 327  ☑ 327 
Q 649+3   T 652  ☑ 652 
Q 58+304  T 362  ☑ 362 
Q 30+404  T 434  ☑ 434 
Q 815+485 T 1300 ☒ 1200
Q 968+692 T 1660 ☒ 1650
Q 118+52  T 170  ☑ 170 
Q 76+101  T 177  ☑ 177 
Q 506+2   T 508 

Q 930+123 T 1053 ☑ 1053
Q 288+98  T 386  ☑ 386 
Q 903+97  T 1000 ☑ 1000
Q 817+490 T 1307 ☑ 1307
Q 871+23  T 894  ☑ 894 
Q 7+465   T 472  ☑ 472 
Q 56+363  T 419  ☑ 419 
Q 662+0   T 662  ☑ 662 
Q 616+59  T 675  ☑ 675 
Q 796+729 T 1525 ☑ 1525
Q 12+676  T 688  ☑ 688 
Q 652+113 T 765  ☑ 765 
Q 879+93  T 972  ☑ 972 
Q 46+229  T 275  ☑ 275 
Q 644+235 T 879  ☑ 879 
Q 651+71  T 722  ☑ 722 
Q 63+20   T 83   ☑ 83  
Q 957+6   T 963  ☑ 963 
Q 302+93  T 395  ☑ 395 
Q 62+572  T 634  ☑ 634 
Q 944+926 T 1870 ☑ 1870
Q 859+271 T 1130 ☒ 1120
Q 52+645  T 697  ☑ 697 
Q 82+990  T 1072 ☑ 1072
Q 9+606   T 615  ☑ 615 
Q 616+602 T 1218 ☑ 1218
Q 577+970 T 1547 ☑ 1547
Q 344+164 T 508  ☑ 508 
Q 951+677 T 1628 ☑ 1628
Q 292+809 T 1101 ☑ 1101
Q 277+78  T 355  ☑ 355 
Q 62+683  T 745  ☑ 745 
Q 17+527  T 544  ☑ 544 
Q 617+76  T 693  ☑ 693 
Q 283+8   T 291  ☑ 291 
Q 381+81  T 462  ☑ 462 
Q 45+905  T 950  ☒ 940 
Q 789+85  T 874  ☑ 874 
Q 99+445  T 544  ☑ 544 
Q 166+55  T 221  ☑ 221 
Q 608+868 T 1476 ☑ 1476
Q 109+80  T 189 

Q 167+189 T 356  ☑ 356 
Q 841+37  T 878  ☑ 878 
Q 906+35  T 941  ☑ 941 
Q 34+706  T 740  ☒ 730 
Q 853+622 T 1475 ☑ 1475
Q 575+51  T 626  ☑ 626 
Q 746+737 T 1483 ☑ 1483
Q 816+27  T 843  ☑ 843 
Q 5+26    T 31   ☑ 31  
Q 9+812   T 821  ☑ 821 
Q 82+403  T 485  ☑ 485 
Q 427+950 T 1377 ☑ 1377
Q 695+8   T 703  ☑ 703 
Q 34+21   T 55   ☑ 55  
Q 713+499 T 1212 ☑ 1212
Q 32+275  T 307  ☑ 307 
Q 44+977  T 1021 ☑ 1021
Q 698+903 T 1601 ☒ 1501
Q 688+464 T 1152 ☑ 1152
Q 50+22   T 72   ☑ 72  
Q 907+599 T 1506 ☒ 1507
Q 316+6   T 322  ☑ 322 
Q 648+860 T 1508 ☑ 1508
Q 5+899   T 904  ☑ 904 
Q 73+167  T 240  ☑ 240 
Q 368+88  T 456  ☑ 456 
Q 683+7   T 690  ☒ 680 
Q 33+70   T 103  ☑ 103 
Q 29+948  T 977  ☑ 977 
Q 97+483  T 580  ☒ 570 
Q 76+8    T 84   ☑ 84  
Q 7+209   T 216  ☑ 216 
Q 238+83  T 321  ☑ 321 
Q 443+287 T 730  ☑ 730 
Q 668+16  T 684  ☑ 684 
Q 260+39  T 299  ☑ 299 
Q 174+951 T 1125 ☑ 1125
Q 676+6   T 682  ☑ 682 
Q 23+700  T 723  ☑ 723 
Q 6+936   T 942  ☑ 942 
Q 94+819  T 913  ☑ 913 
Q 715+108 T 823 

Q 505+211 T 716  ☑ 716 
Q 5+51    T 56   ☒ 55  
Q 745+63  T 808  ☑ 808 
Q 715+925 T 1640 ☒ 1630
Q 334+94  T 428  ☑ 428 
Q 94+548  T 642  ☑ 642 
Q 322+22  T 344  ☑ 344 
Q 426+5   T 431  ☑ 431 
Q 868+44  T 912  ☑ 912 
Q 387+9   T 396  ☑ 396 
Q 217+42  T 259  ☑ 259 
Q 557+941 T 1498 ☑ 1498
Q 54+276  T 330  ☑ 330 
Q 19+525  T 544  ☑ 544 
Q 57+415  T 472  ☑ 472 
Q 84+962  T 1046 ☑ 1046
Q 15+388  T 403  ☑ 403 
Q 70+10   T 80   ☒ 70  
Q 355+67  T 422  ☑ 422 
Q 513+59  T 572  ☑ 572 
Q 63+545  T 608  ☑ 608 
Q 74+443  T 517  ☑ 517 
Q 616+151 T 767  ☑ 767 
Q 535+57  T 592  ☑ 592 
Q 562+717 T 1279 ☑ 1279
Q 643+566 T 1209 ☑ 1209
Q 575+35  T 610  ☑ 610 
Q 414+934 T 1348 ☑ 1348
Q 517+661 T 1178 ☑ 1178
Q 23+917  T 940  ☒ 930 
Q 13+718  T 731  ☑ 731 
Q 16+876  T 892  ☑ 892 
Q 806+187 T 993  ☑ 993 
Q 59+728  T 787  ☑ 787 
Q 85+336  T 421  ☑ 421 
Q 786+928 T 1714 ☑ 1714
Q 798+5   T 803  ☑ 803 
Q 289+95  T 384  ☑ 384 
Q 273+428 T 701  ☑ 701 
Q 980+325 T 1305 ☑ 1305
Q 502+479 T 981  ☑ 981 
Q 847+60  T 907 

...

Q 880+59  T 939  ☑ 939 
Q 60+180  T 240  ☑ 240 
Q 539+30  T 569  ☑ 569 
Q 11+582  T 593  ☑ 593 
Q 978+287 T 1265 ☑ 1265
Q 72+63   T 135  ☑ 135 
Q 3+140   T 143  ☑ 143 
Q 754+507 T 1261 ☑ 1261
Q 702+554 T 1256 ☑ 1256
Q 938+823 T 1761 ☑ 1761
Q 59+941  T 1000 ☒ 9000
Q 87+813  T 900  ☒ 800 
Q 13+447  T 460  ☑ 460 
Q 440+84  T 524  ☑ 524 
Q 87+68   T 155  ☑ 155 
Q 767+96  T 863  ☑ 863 
Q 207+498 T 705  ☒ 606 
Q 263+45  T 308  ☑ 308 
Q 280+170 T 450  ☑ 450 
Q 215+108 T 323  ☑ 323 
Q 891+39  T 930  ☑ 930 
Q 319+6   T 325  ☑ 325 
Q 4+919   T 923  ☑ 923 
Q 524+415 T 939  ☑ 939 
Q 385+50  T 435  ☑ 435 
Q 7+981   T 988  ☑ 988 
Q 673+585 T 1258 ☑ 1258
Q 815+10  T 825  ☑ 825 
Q 61+502  T 563  ☑ 563 
Q 52+89   T 141  ☑ 141 
Q 92+975  T 1067 ☑ 1067
Q 5+658   T 663  ☑ 663 
Q 759+58  T 817  ☑ 817 
Q 34+467  T 501  ☒ 401 
Q 546+32  T 578  ☑ 578 
Q 843+8   T 851  ☑ 851 
Q 158+99  T 257  ☑ 257 
Q 16+356  T 372  ☑ 372 
Q 798+65  T 863  ☑ 863 
Q 761+705 T 1466 ☑ 1466
Q 457+2   T 459  ☒ 469 
Q 69+37   T 106 

Q 759+689 T 1448 ☑ 1448
Q 179+332 T 511  ☑ 511 
Q 623+29  T 652  ☑ 652 
Q 599+963 T 1562 ☑ 1562
Q 31+679  T 710  ☑ 710 
Q 664+143 T 807  ☑ 807 
Q 988+99  T 1087 ☑ 1087
Q 969+7   T 976  ☑ 976 
Q 277+51  T 328  ☑ 328 
Q 69+28   T 97   ☑ 97  
Q 114+67  T 181  ☑ 181 
Q 827+90  T 917  ☑ 917 
Q 77+450  T 527  ☑ 527 
Q 644+7   T 651  ☑ 651 
Q 801+262 T 1063 ☒ 1053
Q 669+0   T 669  ☑ 669 
Q 900+7   T 907  ☑ 907 
Q 904+23  T 927  ☑ 927 
Q 19+275  T 294  ☑ 294 
Q 45+955  T 1000 ☒ 9000
Q 758+96  T 854  ☑ 854 
Q 172+51  T 223  ☑ 223 
Q 76+254  T 330  ☑ 330 
Q 98+205  T 303  ☑ 303 
Q 394+2   T 396  ☑ 396 
Q 0+695   T 695  ☑ 695 
Q 13+535  T 548  ☑ 548 
Q 565+944 T 1509 ☑ 1509
Q 20+2    T 22   ☑ 22  
Q 94+876  T 970  ☑ 970 
Q 223+79  T 302  ☑ 302 
Q 247+273 T 520  ☑ 520 
Q 433+800 T 1233 ☑ 1233
Q 577+321 T 898  ☒ 998 
Q 242+699 T 941  ☑ 941 
Q 776+833 T 1609 ☑ 1609
Q 152+456 T 608  ☒ 508 
Q 765+301 T 1066 ☑ 1066
Q 112+553 T 665  ☑ 665 
Q 516+52  T 568  ☑ 568 
Q 475+175 T 650  ☑ 650 
Q 987+10  T 997 

Q 162+619 T 781  ☑ 781 
Q 844+940 T 1784 ☑ 1784
Q 390+969 T 1359 ☑ 1359
Q 19+10   T 29   ☒ 39  
Q 434+726 T 1160 ☑ 1160
Q 123+42  T 165  ☑ 165 
Q 116+50  T 166  ☑ 166 
Q 4+43    T 47   ☑ 47  
Q 269+57  T 326  ☑ 326 
Q 683+47  T 730  ☑ 730 
Q 661+1   T 662  ☑ 662 
Q 86+152  T 238  ☑ 238 
Q 337+85  T 422  ☑ 422 
Q 55+44   T 99   ☑ 99  
Q 900+198 T 1098 ☒ 1198
Q 73+468  T 541  ☑ 541 
Q 415+941 T 1356 ☑ 1356
Q 594+712 T 1306 ☑ 1306
Q 778+135 T 913  ☑ 913 
Q 451+48  T 499  ☑ 499 
Q 48+87   T 135  ☒ 125 
Q 300+62  T 362  ☑ 362 
Q 23+496  T 519  ☑ 519 
Q 3+638   T 641  ☑ 641 
Q 97+495  T 592  ☑ 592 
Q 30+286  T 316  ☑ 316 
Q 390+5   T 395  ☑ 395 
Q 938+61  T 999  ☑ 999 
Q 713+29  T 742  ☑ 742 
Q 87+537  T 624  ☑ 624 
Q 202+109 T 311  ☑ 311 
Q 48+254  T 302  ☑ 302 
Q 23+962  T 985  ☑ 985 
Q 283+992 T 1275 ☑ 1275
Q 869+16  T 885  ☑ 885 
Q 18+968  T 986  ☑ 986 
Q 946+46  T 992  ☑ 992 
Q 431+34  T 465  ☑ 465 
Q 23+760  T 783  ☑ 783 
Q 8+502   T 510  ☑ 510 
Q 777+76  T 853  ☑ 853 
Q 248+80  T 328 

Q 397+49  T 446  ☑ 446 
Q 550+341 T 891  ☑ 891 
Q 956+453 T 1409 ☑ 1409
Q 370+37  T 407  ☑ 407 
Q 602+15  T 617  ☑ 617 
Q 555+202 T 757  ☑ 757 
Q 299+9   T 308  ☒ 208 
Q 332+405 T 737  ☑ 737 
Q 65+238  T 303  ☑ 303 
Q 99+125  T 224  ☑ 224 
Q 404+97  T 501  ☒ 401 
Q 221+207 T 428  ☑ 428 
Q 196+99  T 295  ☑ 295 
Q 87+526  T 613  ☑ 613 
Q 587+939 T 1526 ☑ 1526
Q 349+22  T 371  ☑ 371 
Q 42+493  T 535  ☑ 535 
Q 935+702 T 1637 ☑ 1637
Q 61+68   T 129  ☑ 129 
Q 190+319 T 509  ☑ 509 
Q 73+907  T 980  ☑ 980 
Q 12+85   T 97   ☑ 97  
Q 91+267  T 358  ☑ 358 
Q 86+487  T 573  ☑ 573 
Q 46+398  T 444  ☑ 444 
Q 823+793 T 1616 ☑ 1616
Q 993+946 T 1939 ☑ 1939
Q 456+274 T 730  ☑ 730 
Q 854+404 T 1258 ☑ 1258
Q 466+534 T 1000 ☑ 1000
Q 580+84  T 664  ☑ 664 
Q 127+235 T 362  ☑ 362 
Q 3+458   T 461  ☑ 461 
Q 114+443 T 557  ☑ 557 
Q 790+487 T 1277 ☑ 1277
Q 46+571  T 617  ☑ 617 
Q 158+239 T 397  ☑ 397 
Q 325+23  T 348  ☑ 348 
Q 56+797  T 853  ☑ 853 
Q 557+38  T 595  ☑ 595 
Q 25+66   T 91   ☑ 91  
Q 60+234  T 294 

Q 891+973 T 1864 ☑ 1864
Q 22+628  T 650  ☑ 650 
Q 80+225  T 305  ☑ 305 
Q 361+727 T 1088 ☑ 1088
Q 665+20  T 685  ☑ 685 
Q 945+62  T 1007 ☑ 1007
Q 9+699   T 708  ☒ 608 
Q 378+319 T 697  ☑ 697 
Q 0+884   T 884  ☑ 884 
Q 987+45  T 1032 ☑ 1032
Q 4+597   T 601  ☑ 601 
Q 186+822 T 1008 ☒ 1098
Q 52+201  T 253  ☑ 253 
Q 50+749  T 799  ☑ 799 
Q 495+393 T 888  ☑ 888 
Q 664+73  T 737  ☑ 737 
Q 73+2    T 75   ☑ 75  
Q 36+86   T 122  ☑ 122 
Q 781+44  T 825  ☑ 825 
Q 463+82  T 545  ☑ 545 
Q 698+401 T 1099 ☒ 1199
Q 483+671 T 1154 ☑ 1154
Q 10+13   T 23   ☒ 20  
Q 79+22   T 101  ☑ 101 
Q 684+20  T 704  ☑ 704 
Q 677+25  T 702  ☑ 702 
Q 366+949 T 1315 ☑ 1315
Q 72+456  T 528  ☑ 528 
Q 254+59  T 313  ☑ 313 
Q 505+295 T 800  ☒ 700 
Q 61+774  T 835  ☑ 835 
Q 217+4   T 221  ☑ 221 
Q 967+678 T 1645 ☑ 1645
Q 65+948  T 1013 ☑ 1013
Q 886+999 T 1885 ☑ 1885
Q 82+332  T 414  ☑ 414 
Q 670+99  T 769  ☑ 769 
Q 381+462 T 843  ☑ 843 
Q 71+110  T 181  ☑ 181 
Q 31+566  T 597  ☑ 597 
Q 400+79  T 479  ☑ 479 
Q 168+804 T 972 